# Supply Trend on fixture date

## Voyages Data & HTL joint Use Case

The following Use Case serves the need of monitoring the supply trend when a fixture is closed. We are going to use the voyages data api in order to retrieve the voyages of interest. Then, using the voyages data, we are going to query the HTL api and get the supply trend at the point in time that each vessel was fixed.  
For the sake of the example we are going to focus on voyages within 2021 having as commercial operator Vitol.

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
#pip install signal-ocean

In [2]:
import pandas as pd
from datetime import date, timedelta,time

# imports regarding HTL
from signal_ocean import PortAPI, VesselClassAPI,PortFilter,VesselClassFilter,Connection
from signal_ocean.historical_tonnage_list import VesselFilter, PushType, MarketDeployment, CommercialStatus, VesselSubclass
from signal_ocean.historical_tonnage_list import HistoricalTonnageListAPI
from signal_ocean.historical_tonnage_list import Column, IndexLevel

# imports regarding Voyages Data
from signal_ocean.voyages import VoyagesAPI

In [3]:
signal_ocean_api_key = '' #replace with your subscription key
connection = Connection(signal_ocean_api_key)

In [4]:
# instantiating connections for all necessary apis 
port_api = PortAPI(connection)
vessel_class_api = VesselClassAPI(connection)
voyages_api = VoyagesAPI(connection)
htl_api = HistoricalTonnageListAPI(connection)

### Getting the voyages

In [5]:
vessel_class = vessel_class_api.get_vessel_classes(VesselClassFilter(name_like='aframax'))[0]
vessel_class_id = vessel_class.id

start_date = date(2021,1,1)

In [6]:
voyages = voyages_api.get_voyages(vessel_class_id=vessel_class_id, date_from=start_date)
voyages_df = pd.DataFrame([v.__dict__ for v in voyages])

In [7]:
#list of all Commercial Operators found in the dataset
voyages_df.commercial_operator.unique()

array(['Pertamina', 'Equinor', 'Elmira Tankers', 'PDVSA', 'Saud Shipping',
       'International Marine Management', 'ECB International',
       'Dynacom Tankers', 'Norstar Shipping', 'Knutsen',
       'Olympic Shipping & Management', 'Torm', 'Eurotankers',
       'Centrofin', 'Sanmar Shipping', 'Fareast Shipmanagement',
       'Soechi Lines', 'Sovcomflot', 'Unknown', 'Bihar International',
       'Chandris', 'ST Shipping & Transport', 'Red Sea Marine Services',
       'PT Buana Lintas Lautan Tbk', 'Silk Road Petroleum',
       'Gerhard Shipping', 'Jellicoe Tankers', 'NITC', 'Nathalin',
       'Zodiac Maritime', 'Marshal Shipping', 'Petronas', 'Navig8 group',
       'Shanghai Zhenhua Shipping', 'Cakra Manunggal Semesta PT',
       'Eastern Mediterranean Maritime', 'Pelayaran Sakti Erawan',
       'Teekay Corp', 'Coral Shipping', 'PetroVietnam',
       'Rosewood Shipping', 'Darya Shipping', 'Monte Nero Maritime',
       'New Shipping', 'Berlian Laju Tanker', 'Thenamaris',
       'Genera

In [8]:
# we need the voyages for which we know the fixture date
com_op_voyages = voyages_df[(voyages_df.commercial_operator=='Vitol') & \
                           (voyages_df.fixture_date.notna())].copy()

In [9]:
com_op_voyages.tail()

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold,is_implied_by_ais,has_manual_entries,ballast_distance,laden_distance
4545,9821706,24,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED7805D00', port_id=3...",I95DE0AVED7805D00,Elandra Sound,...,2021-01-11 19:55:25+00:00,1.0,FullyFixed,2021-01-05 14:26:27+00:00,False,False,None,None,549.77,5311.35
4547,9821706,26,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED7C24800', port_id=1...",I95DE0AVED7C24800,Elandra Sound,...,2021-02-23 19:56:24+00:00,1.0,FullyFixed,2021-02-15 12:23:18+00:00,True,False,None,None,466.73,554.81
4550,9821706,29,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED8116200', port_id=3...",I95DE0AVED8116200,Elandra Sound,...,2021-05-12 00:00:00+00:00,1.0,FullyFixed,2021-04-27 09:44:15+00:00,False,False,None,True,1691.13,1979.97
4551,9821706,30,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED81E9100', port_id=3...",I95DE0AVED81E9100,Elandra Sound,...,2021-06-02 00:00:00+00:00,0.0,OnSubs,2021-05-19 11:20:25.850000+00:00,False,False,None,True,NaN,NaN
4605,9829899,20,1,86,1,1831,False,"(VoyageEvent(id='I95FE0BSED7CF7700', port_id=3...",I95FE0BVED7CF7700,Navig8 Pride Lhj,...,2021-03-26 23:59:00+00:00,1.0,FullyFixed,2021-03-12 02:44:42+00:00,False,False,None,None,2605.56,2874.46


We need the information regarding the first load of the voyage. It will be the basis port for the htl query.

In [10]:
def get_voyage_first_load_port(voyage_events):
    return next((e.port_name for e in voyage_events or [] if e.purpose=='Load'), None)

com_op_voyages['first_load_port'] = com_op_voyages['events'].apply(get_voyage_first_load_port)

The htl_query function is going to call the HTL api for each voyage, using as parameters its attributes.  
Also it should be mentioned that fixture date refers to the date of the first fixture reporting the specific voyage.   

In [11]:
import numpy as np

def htl_query(**params):
    port = port_api.get_ports(PortFilter(name_like=params['port_name']))[0]
    vessel_class = vessel_class_api.get_vessel_classes(VesselClassFilter(name_like=params['vessel_class']))[0]
    date_of_interest = params['fixture_date']
    laycanEndInDays = (params['laycan_end'] - date_of_interest).days
    
    if laycanEndInDays<=0:
        return {}
    
    vessel_filter = VesselFilter(
        push_types=[PushType.PUSHED, PushType.PUSHED_POSS],
        market_deployments=[MarketDeployment.RELET, MarketDeployment.SPOT],
        commercial_statuses=[CommercialStatus.AVAILABLE, CommercialStatus.CANCELLED, CommercialStatus.FAILED],
        vessel_subclass=VesselSubclass.DIRTY,
        latest_ais_since=5
    )
    
    htl_for_supply_trend = htl_api.get_historical_tonnage_list(
        port,
        vessel_class,
        laycanEndInDays,
        start_date = date_of_interest- timedelta(days=3),
        end_date = date_of_interest,
        vessel_filter=vessel_filter,
        time=time(hour=6)
    )
    supply_trend_data_frame = htl_for_supply_trend.to_data_frame()
    supply_trend = supply_trend_data_frame.groupby(IndexLevel.DATE, sort=True).size().to_dict()
    
    return supply_trend
    

In [12]:
print(f'# of calls to HTL api: {com_op_voyages.shape[0]}')

# of calls to HTL api: 44


In [13]:
com_op_voyages['supply_trend'] = com_op_voyages\
                                .apply(lambda row:htl_query(port_name = row['first_load_port'],
                                                            vessel_class = row['vessel_class'],
                                                            fixture_date = row['fixture_date'].date(),
                                                            laycan_end = row['laycan_to'].date()),axis=1)

In [14]:
com_op_voyages[['Same day','1 day before','2 days before','3 days before']] = com_op_voyages['supply_trend']\
                                                                             .apply(lambda x: list(x.values())[::-1]).apply(pd.Series).values

In [15]:
com_op_voyages['laycan window'] = [str(laycan_window.days) + ' days' for laycan_window in  (com_op_voyages['laycan_to'] - com_op_voyages['fixture_date'])]

In [16]:
supply_trend_on_fixtures = com_op_voyages[['vessel_name','voyage_number','fixture_date','first_load_port',
                                           'laycan window','Same day','1 day before','2 days before','3 days before']].dropna()

In [17]:
supply_trend_on_fixtures = supply_trend_on_fixtures.astype({'fixture_date':'datetime64[ns]'})
supply_trend_on_fixtures.sort_values('fixture_date').head()

,vessel_name,voyage_number,fixture_date,first_load_port,laycan window,Same day,1 day before,2 days before,3 days before
4545,Elandra Sound,24,2021-01-05 14:26:27.000,Corpus Christi,6 days,7,13,10,10
4198,Antonis,54,2021-01-05 18:03:19.247,Cayo Arcas,9 days,14,17,20,18
4045,Dong-A Thetis,33,2021-01-11 16:29:28.000,Yanbu,17 days,131,122,115,130
1572,Alhani,102,2021-01-11 16:53:43.003,Corpus Christi,8 days,23,20,19,19
4048,Dong-A Maia,50,2021-01-11 21:05:10.203,Corpus Christi,10 days,27,25,23,19


In [18]:
supply_trend_on_fixtures.to_excel('supply_trend_on_fixture_day.xlsx')